In [29]:
import os
import shutil
import pandas as pd
import sqlite3

for folder in ["raw", "processed", "output"]:
    os.makedirs(folder, exist_ok=True)


shutil.copy("/content/retail_sales_dataset.csv", "raw/retail_sales_dataset.csv")


df = pd.read_csv("raw/retail_sales_dataset.csv")
print("Raw rows:", len(df))
df.head()

df = df.drop_duplicates()

df = df.fillna({
    "Profit": 0,
    "Sales": 0,
    "Category": "Unknown"
})

df.columns = df.columns.str.lower().str.replace(" ", "_")

df.rename(columns={"total_amount":"sales"}, inplace=True)

#print(df.columns.tolist())
df["profit"] = df["sales"] * 0.25

df["sales"] = df["sales"].astype(float)
df["profit"] = df["profit"].astype(float)
df["quantity"] = df["quantity"].astype(int)
df["age"] = df["age"].astype(int)

df["margin"] = df["profit"] / df["sales"]

df["high_value_sale"] = df["sales"].apply(lambda x: 1 if x > 500 else 0)

df["senior_customer"] = df["age"].apply(lambda x: 1 if x >= 60 else 0)

customers = df[["customer_id","gender","age","senior_customer"]].drop_duplicates()

orders = df[["transaction_id","date","sales","profit","margin","high_value_sale"]]

products = df[["product_category","price_per_unit"]].drop_duplicates()

df.to_csv("processed/processed_data.csv", index=False)

customers.to_csv("output/customers.csv", index=False)
orders.to_csv("output/orders.csv", index=False)
products.to_csv("output/products.csv", index=False)

#import sqlite3

conn = sqlite3.connect("database.sqlite")

customers.to_sql("customers", conn, if_exists="replace", index=False)
orders.to_sql("orders", conn, if_exists="replace", index=False)
products.to_sql("products", conn, if_exists="replace", index=False)

conn.close()

print("Processed rows:", len(df))
print("Customers:", len(customers))
print("Orders:", len(orders))
print("Products:", len(products))


Raw rows: 1000
Processed rows: 1000
Customers: 1000
Orders: 1000
Products: 15
